# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 9 2022 12:00AM,239178,10,8091243,Eywa Pharma Inc.,Released
1,Jun 9 2022 12:00AM,239178,10,7999842-BO,Eywa Pharma Inc.,Released
2,Jun 9 2022 12:00AM,239178,10,8091236-BO,Eywa Pharma Inc.,Released
3,Jun 9 2022 12:00AM,239178,10,8091238-BO,Eywa Pharma Inc.,Released
4,Jun 9 2022 12:00AM,239178,10,8093052-BO,Eywa Pharma Inc.,Released
5,Jun 9 2022 12:00AM,239178,10,8114985,Eywa Pharma Inc.,Released
6,Jun 9 2022 12:00AM,239178,10,8114986,Eywa Pharma Inc.,Released
7,Jun 9 2022 12:00AM,239178,10,8114987,Eywa Pharma Inc.,Released
8,Jun 9 2022 12:00AM,239178,10,8117644,Eywa Pharma Inc.,Released
9,Jun 9 2022 12:00AM,239178,10,8117647,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
62,239175,Released,1
63,239176,Released,1
64,239178,Released,27
65,239179,Released,2
66,239180,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
239175,NaN,NaN,NaN,1.0
239176,NaN,NaN,NaN,1.0
239178,NaN,NaN,NaN,27.0
239179,NaN,NaN,NaN,2.0
239180,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
239039,0.0,0.0,0.0,2.0
239045,0.0,29.0,3.0,0.0
239060,0.0,25.0,9.0,1.0
239066,0.0,0.0,0.0,1.0
239070,0.0,3.0,13.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
239039,0.0,0,0,2
239045,0.0,29,3,0
239060,0.0,25,9,1
239066,0.0,0,0,1
239070,0.0,3,13,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,239039,0.0,0,0,2
1,239045,0.0,29,3,0
2,239060,0.0,25,9,1
3,239066,0.0,0,0,1
4,239070,0.0,3,13,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,239039,0.0,,,2
1,239045,0.0,29,3,
2,239060,0.0,25,9,1
3,239066,0.0,,,1
4,239070,0.0,3,13,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 9 2022 12:00AM,239178,10,Eywa Pharma Inc.
27,Jun 9 2022 12:00AM,239171,10,Emerginnova
30,Jun 9 2022 12:00AM,239173,10,Yusen – 3D Matrix
31,Jun 9 2022 12:00AM,239175,10,Yusen – 3D Matrix
32,Jun 9 2022 12:00AM,239174,10,Cosmo International Fragrances
33,Jun 8 2022 3:55PM,239180,21,"NBTY Global, Inc."
34,Jun 8 2022 3:54PM,239179,12,Hush Hush
36,Jun 8 2022 3:41PM,239176,21,"NBTY Global, Inc."
37,Jun 8 2022 3:34PM,239172,19,"GCH Granules USA, Inc."
38,Jun 8 2022 3:32PM,239170,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Jun 9 2022 12:00AM,239178,10,Eywa Pharma Inc.,0.0,,,27
1,Jun 9 2022 12:00AM,239171,10,Emerginnova,0.0,,,3
2,Jun 9 2022 12:00AM,239173,10,Yusen – 3D Matrix,0.0,,,1
3,Jun 9 2022 12:00AM,239175,10,Yusen – 3D Matrix,0.0,,,1
4,Jun 9 2022 12:00AM,239174,10,Cosmo International Fragrances,1.0,,,
5,Jun 8 2022 3:55PM,239180,21,"NBTY Global, Inc.",0.0,,,1
6,Jun 8 2022 3:54PM,239179,12,Hush Hush,0.0,,,2
7,Jun 8 2022 3:41PM,239176,21,"NBTY Global, Inc.",0.0,,,1
8,Jun 8 2022 3:34PM,239172,19,"GCH Granules USA, Inc.",0.0,,,1
9,Jun 8 2022 3:32PM,239170,21,"NBTY Global, Inc.",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 12:00AM,239178,10,Eywa Pharma Inc.,27,,
1,Jun 9 2022 12:00AM,239171,10,Emerginnova,3,,
2,Jun 9 2022 12:00AM,239173,10,Yusen – 3D Matrix,1,,
3,Jun 9 2022 12:00AM,239175,10,Yusen – 3D Matrix,1,,
4,Jun 9 2022 12:00AM,239174,10,Cosmo International Fragrances,,,
5,Jun 8 2022 3:55PM,239180,21,"NBTY Global, Inc.",1,,
6,Jun 8 2022 3:54PM,239179,12,Hush Hush,2,,
7,Jun 8 2022 3:41PM,239176,21,"NBTY Global, Inc.",1,,
8,Jun 8 2022 3:34PM,239172,19,"GCH Granules USA, Inc.",1,,
9,Jun 8 2022 3:32PM,239170,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 12:00AM,239178,10,Eywa Pharma Inc.,27,,
1,Jun 9 2022 12:00AM,239171,10,Emerginnova,3,,
2,Jun 9 2022 12:00AM,239173,10,Yusen – 3D Matrix,1,,
3,Jun 9 2022 12:00AM,239175,10,Yusen – 3D Matrix,1,,
4,Jun 9 2022 12:00AM,239174,10,Cosmo International Fragrances,,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 12:00AM,239178,10,Eywa Pharma Inc.,27.0,NaN,NaN
1,Jun 9 2022 12:00AM,239171,10,Emerginnova,3.0,NaN,NaN
2,Jun 9 2022 12:00AM,239173,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Jun 9 2022 12:00AM,239175,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Jun 9 2022 12:00AM,239174,10,Cosmo International Fragrances,NaN,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 12:00AM,239178,10,Eywa Pharma Inc.,27.0,0.0,0.0
1,Jun 9 2022 12:00AM,239171,10,Emerginnova,3.0,0.0,0.0
2,Jun 9 2022 12:00AM,239173,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Jun 9 2022 12:00AM,239175,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Jun 9 2022 12:00AM,239174,10,Cosmo International Fragrances,0.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4304630,88.0,30.0,15.0
102,478176,2.0,0.0,0.0
12,478262,3.0,0.0,0.0
15,239045,0.0,3.0,29.0
16,956566,3.0,1.0,0.0
18,239107,1.0,0.0,0.0
19,1673913,22.0,38.0,25.0
20,1434780,33.0,19.0,4.0
21,1674122,6.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4304630,88.0,30.0,15.0
1,102,478176,2.0,0.0,0.0
2,12,478262,3.0,0.0,0.0
3,15,239045,0.0,3.0,29.0
4,16,956566,3.0,1.0,0.0
5,18,239107,1.0,0.0,0.0
6,19,1673913,22.0,38.0,25.0
7,20,1434780,33.0,19.0,4.0
8,21,1674122,6.0,1.0,0.0
9,22,478297,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,88.0,30.0,15.0
1,102,2.0,0.0,0.0
2,12,3.0,0.0,0.0
3,15,0.0,3.0,29.0
4,16,3.0,1.0,0.0
5,18,1.0,0.0,0.0
6,19,22.0,38.0,25.0
7,20,33.0,19.0,4.0
8,21,6.0,1.0,0.0
9,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,102,Released,2.0
2,12,Released,3.0
3,15,Released,0.0
4,16,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21,22
Status,,,,,,,,,,
Completed,15.0,0.0,0.0,29.0,0.0,0.0,25.0,4.0,0.0,0.0
Executing,30.0,0.0,0.0,3.0,1.0,0.0,38.0,19.0,1.0,0.0
Released,88.0,2.0,3.0,0.0,3.0,1.0,22.0,33.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21,22
0,Completed,15.0,0.0,0.0,29.0,0.0,0.0,25.0,4.0,0.0,0.0
1,Executing,30.0,0.0,0.0,3.0,1.0,0.0,38.0,19.0,1.0,0.0
2,Released,88.0,2.0,3.0,0.0,3.0,1.0,22.0,33.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21,22
0,Completed,15.0,0.0,0.0,29.0,0.0,0.0,25.0,4.0,0.0,0.0
1,Executing,30.0,0.0,0.0,3.0,1.0,0.0,38.0,19.0,1.0,0.0
2,Released,88.0,2.0,3.0,0.0,3.0,1.0,22.0,33.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()